In [ ]:
# ============================================
# 批量计算 8 个栅格的 Z-Score 标准化 (ArcPy)
# --------------------------------------------
# 输入路径: 原始 90m 栅格
# 输出路径: 每个波段的 z-score 栅格 (_zscore.tif)
# ============================================

import arcpy
from arcpy.sa import *
import os

# 启用 Spatial Analyst 扩展
arcpy.CheckOutExtension("Spatial")

# ---------------------------
# 1?? 输入输出路径
# ---------------------------
in_folder = r"F:\00.博后研究\城乡连续体制图\New_NEW\00.Data\Landsat_NL_90m_2010\Landsat_NL_Mector_90m"
out_folder = r"F:\00.博后研究\城乡连续体制图\New_NEW\00.Data\Landsat_NL_90m_2010\Landsat_NL_Mector_90m_zscore"

# 若输出文件夹不存在则创建
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

# ---------------------------
# 2?? 需要处理的 8 个栅格文件名
# ---------------------------
raster_names = [
    "Landsat_BLUE_2010_90m.tif",
    "Landsat_GREEN_2010_90m.tif",
    "Landsat_NIR_2010_90m.tif",
    "Landsat_RED_2010_90m.tif",
    "Landsat_SWIR1_2010_90m.tif",
    "Landsat_SWIR2_2010_90m.tif",
    "Landsat_TEMP1_2010_90m.tif",
    "DMSP_2010_90m.tif"
]

# ---------------------------
# 3?? 循环处理每个栅格
# ---------------------------
for name in raster_names:
    in_ras = os.path.join(in_folder, name)
    print("Processing:", in_ras)
    
    # 读取均值和标准差
    mean_val = float(arcpy.GetRasterProperties_management(in_ras, "MEAN").getOutput(0))
    std_val  = float(arcpy.GetRasterProperties_management(in_ras, "STD").getOutput(0))
    
    # 避免标准差为0导致除零错误
    if std_val == 0:
        raise ValueError(f"Raster {name} 的标准差为0，无法计算 z-score。")
    
    # 计算 Z-score
    z_ras = (Raster(in_ras) - mean_val) / std_val
    
    # 输出文件路径
    out_name = os.path.splitext(name)[0] + "_zscore.tif"
    out_path = os.path.join(out_folder, out_name)
    
    # 保存结果
    z_ras.save(out_path)
    print(f"? 已保存: {out_path}")

print("?? 所有波段已完成 z-score 标准化！")
arcpy.CheckInExtension("Spatial")